In [15]:
import numpy as np
import pandas as pd
import json

In [16]:
data = pd.read_csv("/home/armin/Documents/HTVS_dev/analysis_notebooks/res_csv/raw_data.csv")
data.tail(10)

,PCC,Round,Source,PCC Check,FEN_Loc,FEN,DEC_Loc,DEC,Logged,Notes
143,SGEGR,0.0,qNHVEI,Yes,cds3,Yes,cds3,Yes,Yes,NaN
144,GRGFG,0.0,qNHVEI,Yes,cds3,Yes,cds3,Yes,Yes,NaN
145,GHGGF,0.0,qNHVEI,Yes,cds3,Yes,cds3,Yes,Yes,NaN
146,GGDEK,0.0,qNHVEI,Yes,cds3,Yes,cds3,Yes,Yes,NaN
147,LGGNG,0.0,qNHVEI,Yes,cds3,Yes,cds3,Yes,Yes,NaN
148,KGGED,0.0,qNHVEI,Yes,cds3,Yes,cds3,Yes,Yes,NaN
149,FGGDG,0.0,qNHVEI,Yes,cds3,Yes,cds3,Yes,Yes,NaN
150,YGYNT,NaN,qNHVEI,No,NaN,No,NaN,No,No,NaN
151,YGTYT,NaN,qNHVEI,No,NaN,No,NaN,No,No,NaN
152,YGYHE,NaN,qNHVEI,No,NaN,No,NaN,No,No,NaN


In [17]:
data.drop([150, 151, 152], inplace=True)
data.drop(columns=["PCC Check", "FEN_Loc", "DEC_Loc", "Logged", "FEN", "DEC"], inplace=True)
data = data.astype({'Round': 'int8'})
data.rename(columns={"Round": "round", "Source": "acqf", "Notes": "phase"}, inplace=True)
data.tail(10)

,PCC,round,acqf,phase
140,GWGHG,0,qNHVEI,NaN
141,GTGSG,0,qNHVEI,NaN
142,GHASF,0,qNHVEI,NaN
143,SGEGR,0,qNHVEI,NaN
144,GRGFG,0,qNHVEI,NaN
145,GHGGF,0,qNHVEI,NaN
146,GGDEK,0,qNHVEI,NaN
147,LGGNG,0,qNHVEI,NaN
148,KGGED,0,qNHVEI,NaN
149,FGGDG,0,qNHVEI,NaN


In [18]:
for row in data[data["phase"] == "Filtered W and Y"].iterrows():
    data.loc[row[0], "phase"] = 4

for row in data[data["phase"] == "Filtered W"].iterrows():
    data.loc[row[0], "phase"] = 3

for row in data[(data["round"] <= 15) & (data["round"] >= 7)].iterrows():
    data.loc[row[0], "phase"] = 2

for row in data[(data["round"] <= 6) & (data["round"] >= 1)].iterrows():
    data.loc[row[0], "phase"] = 1

for row in data[(data["round"] == 0)].iterrows():
    data.loc[row[0], "phase"] = 0
    data.loc[row[0], "acqf"] = "init"

In [19]:
data["dG_FEN"] = np.zeros((150, ))
data["dG_FEN_err"] = np.zeros((150, ))
data["dG_ACT"] = np.zeros((150, ))
data["dG_ACT_err"] = np.zeros((150, ))

In [20]:
reports_dir = "/home/armin/Documents/HTVS_dev/results/"
ind_dict = {row[1].PCC: row[0] for row in data.iterrows()}
for pcc in data.PCC.tolist():
    with open(reports_dir+f"{pcc}_FEN.JSON") as f:
        fen_data = json.load(f)
    data.loc[ind_dict[pcc], "dG_FEN"] = round(fen_data["FE"], 2)
    data.loc[ind_dict[pcc], "dG_FEN_err"] = round(fen_data["FE_error"], 2)
    with open(reports_dir+f"{pcc}_DEC.JSON") as f:
        dec_data = json.load(f)
    data.loc[ind_dict[pcc], "dG_ACT"] = round(dec_data["FE"], 2)
    data.loc[ind_dict[pcc], "dG_ACT_err"] = round(dec_data["FE_error"], 2)

In [21]:
data.head()

,PCC,round,acqf,phase,dG_FEN,dG_FEN_err,dG_ACT,dG_ACT_err
0,FHVPY,20,qUCB,4,-7.61,0.29,-7.11,0.65
1,GFHVY,20,qUCB,4,-7.88,0.18,-5.76,0.23
2,HHWTW,20,qUCB,4,-7.49,0.41,-6.24,0.17
3,FTFGF,20,qNHVEI,4,-7.09,0.17,-5.41,0.16
4,FTHGF,20,qNHVEI,4,-7.00,0.27,-5.54,0.11


In [22]:
data["FEN_sen"] = -data.dG_FEN
data["FEN_sen_err"] = data.dG_FEN_err.copy()
data["FEN_sel"] = data.dG_ACT - data.dG_FEN
data["FEN_sel_err"] = round(np.sqrt(data.dG_FEN_err**2 + data.dG_ACT_err**2), 2)
data["DEC_sen"] = -data.dG_ACT
data["DEC_sen_err"] = data.dG_ACT_err.copy()
A = 1/(6.022e23*5**3*1e-24)
B = 1000/8.314/300
data["K_d_FEN"] = round(A*np.exp(B*data.dG_FEN)*1e6, 2)
data["K_d_FEN_err"] = round(B*data.K_d_FEN*data.dG_FEN_err, 2)
data["K_d_ACT"] = round(A*np.exp(B*data.dG_ACT)*1e6, 2)
data["K_d_ACT_err"] = round(B*data.K_d_ACT*data.dG_ACT_err, 2)

In [23]:
data.head()

,PCC,round,acqf,phase,dG_FEN,dG_FEN_err,dG_ACT,dG_ACT_err,FEN_sen,FEN_sen_err,FEN_sel,FEN_sel_err,DEC_sen,DEC_sen_err,K_d_FEN,K_d_FEN_err,K_d_ACT,K_d_ACT_err
0,FHVPY,20,qUCB,4,-7.61,0.29,-7.11,0.65,7.61,0.29,0.50,0.71,7.11,0.65,628.47,73.07,767.97,200.14
1,GFHVY,20,qUCB,4,-7.88,0.18,-5.76,0.23,7.88,0.18,2.12,0.29,5.76,0.23,563.99,40.70,1319.50,121.68
2,HHWTW,20,qUCB,4,-7.49,0.41,-6.24,0.17,7.49,0.41,1.25,0.44,6.24,0.17,659.44,108.40,1088.50,74.19
3,FTFGF,20,qNHVEI,4,-7.09,0.17,-5.41,0.16,7.09,0.17,1.68,0.23,5.41,0.16,774.15,52.76,1518.28,97.40
4,FTHGF,20,qNHVEI,4,-7.00,0.27,-5.54,0.11,7.00,0.27,1.46,0.29,5.54,0.11,802.60,86.88,1441.17,63.56


In [24]:
exdata = {"PCC": ["FGWWW",
                "HHWWW",
                "HWGWW",
                "HWWTW",
                "WGWWW",
                "WWWWY",
                "YGAWW",
                "YGWWW",
                "YNAWW"
],
        "dG_FEN_extended" : ["-9.65",
                "-10.43",
                "-9.06 ",
                "-9.21 ",
                "-9.56 ",
                "-10.30",
                "-8.41 ",
                "-11.14",
                "-10.09"
        ],
        "dG_FEN_extended_err": ["0.35",
                "0.32",
                "0.35",
                "0.37",
                "0.47",
                "0.46",
                "0.17",
                "0.53",
                "0.26"
        ], 
        "dG_ACT_extended": ["-6.80",
                "-6.40",
                "-6.53",
                "-6.53",
                "-6.18",
                "-6.61",
                "-5.83",
                "-6.43",
                "-6.55"
        ],
        "dG_ACT_extended_err": [
                "0.32",
                "0.27",
                "0.52",
                "0.20",
                "0.19",
                "0.16",
                "0.49",
                "0.16",
                "0.25"
        ]
}
exdata = pd.DataFrame(exdata)
exdata.dG_FEN_extended = exdata.dG_FEN_extended.astype(float)
exdata.dG_FEN_extended_err = exdata.dG_FEN_extended_err.astype(float)
exdata.dG_ACT_extended = exdata.dG_ACT_extended.astype(float)
exdata.dG_ACT_extended_err = exdata.dG_ACT_extended_err.astype(float)
exdata["FEN_sen_extended"] = -exdata.dG_FEN_extended.copy()
exdata["FEN_sen_extended_err"] = exdata.dG_FEN_extended_err.copy()
exdata["FEN_spe_extended"] = exdata.dG_ACT_extended - exdata.dG_FEN_extended
exdata["FEN_spe_extended_err"] = round(np.sqrt(exdata.dG_FEN_extended_err**2 + exdata.dG_ACT_extended_err**2), 2)
exdata.head(10)

,PCC,dG_FEN_extended,dG_FEN_extended_err,dG_ACT_extended,dG_ACT_extended_err,FEN_sen_extended,FEN_sen_extended_err,FEN_spe_extended,FEN_spe_extended_err
0,FGWWW,-9.65,0.35,-6.80,0.32,9.65,0.35,2.85,0.47
1,HHWWW,-10.43,0.32,-6.40,0.27,10.43,0.32,4.03,0.42
2,HWGWW,-9.06,0.35,-6.53,0.52,9.06,0.35,2.53,0.63
3,HWWTW,-9.21,0.37,-6.53,0.20,9.21,0.37,2.68,0.42
4,WGWWW,-9.56,0.47,-6.18,0.19,9.56,0.47,3.38,0.51
5,WWWWY,-10.30,0.46,-6.61,0.16,10.30,0.46,3.69,0.49
6,YGAWW,-8.41,0.17,-5.83,0.49,8.41,0.17,2.58,0.52
7,YGWWW,-11.14,0.53,-6.43,0.16,11.14,0.53,4.71,0.55
8,YNAWW,-10.09,0.26,-6.55,0.25,10.09,0.26,3.54,0.36


In [25]:
data = data.merge(exdata, on='PCC', how='left')

In [26]:
data["K_d_FEN_extended"] =     round(A*np.exp(B*data.dG_FEN_extended)*1e6, 2)
data["K_d_FEN_extended_err"] = round(B*data.K_d_FEN_extended*data.dG_FEN_extended_err, 2)
data["K_d_ACT_extended"] =     round(A*np.exp(B*data.dG_ACT_extended)*1e6, 2)
data["K_d_ACT_extended_err"] = round(B*data.K_d_ACT_extended*data.dG_ACT_extended_err, 2)
data.head(10)

,PCC,round,acqf,phase,dG_FEN,dG_FEN_err,dG_ACT,dG_ACT_err,FEN_sen,FEN_sen_err,...,dG_ACT_extended,dG_ACT_extended_err,FEN_sen_extended,FEN_sen_extended_err,FEN_spe_extended,FEN_spe_extended_err,K_d_FEN_extended,K_d_FEN_extended_err,K_d_ACT_extended,K_d_ACT_extended_err
0,FHVPY,20,qUCB,4,-7.61,0.29,-7.11,0.65,7.61,0.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GFHVY,20,qUCB,4,-7.88,0.18,-5.76,0.23,7.88,0.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HHWTW,20,qUCB,4,-7.49,0.41,-6.24,0.17,7.49,0.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FTFGF,20,qNHVEI,4,-7.09,0.17,-5.41,0.16,7.09,0.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FTHGF,20,qNHVEI,4,-7.00,0.27,-5.54,0.11,7.00,0.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,FTTGF,20,qNHVEI,4,-6.68,0.20,-5.72,0.22,6.68,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,HTFHE,19,qNHVEI,4,-5.08,0.20,-5.50,0.10,5.08,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NFHVH,19,qNHVEI,4,-7.30,0.25,-9.59,1.14,7.30,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,HFHNT,19,qNHVEI,4,-7.33,0.33,-5.50,0.21,7.33,0.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,YTLYP,19,qUCB,4,-8.42,0.37,-6.02,0.21,8.42,0.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
data[data.dG_ACT_extended.notna()]

,PCC,round,acqf,phase,dG_FEN,dG_FEN_err,dG_ACT,dG_ACT_err,FEN_sen,FEN_sen_err,...,dG_ACT_extended,dG_ACT_extended_err,FEN_sen_extended,FEN_sen_extended_err,FEN_spe_extended,FEN_spe_extended_err,K_d_FEN_extended,K_d_FEN_extended_err,K_d_ACT_extended,K_d_ACT_extended_err
30,YNAWW,15,qNHVEI,2,-10.48,0.28,-6.08,0.25,10.48,0.28,...,-6.55,0.25,10.09,0.26,3.54,0.36,232.52,24.24,961.28,96.35
37,YGAWW,14,qNHVEI,2,-8.56,0.23,-4.36,0.11,8.56,0.23,...,-5.83,0.49,8.41,0.17,2.58,0.52,456.02,31.08,1282.98,252.05
55,FGWWW,11,qNHVEI,2,-10.76,0.43,-7.31,0.47,10.76,0.43,...,-6.80,0.32,9.65,0.35,2.85,0.47,277.38,38.92,869.60,111.57
56,WGWWW,11,qNHVEI,2,-11.06,0.49,-7.01,0.20,11.06,0.49,...,-6.18,0.19,9.56,0.47,3.38,0.51,287.57,54.19,1115.00,84.94
69,YGWWW,9,qNHVEI,2,-12.46,0.47,-6.26,0.19,12.46,0.47,...,-6.43,0.16,11.14,0.53,4.71,0.55,152.63,32.43,1008.66,64.70
70,HWGWW,9,qNHVEI,2,-9.91,0.54,-5.90,0.23,9.91,0.54,...,-6.53,0.52,9.06,0.35,2.53,0.63,351.40,49.31,969.02,202.02
96,HWWTW,3,qNHVEI,1,-10.43,0.48,-6.52,0.28,10.43,0.48,...,-6.53,0.20,9.21,0.37,2.68,0.42,330.89,49.09,969.02,77.70
106,HHWWW,0,init,0,-10.47,0.38,-7.39,0.15,10.47,0.38,...,-6.40,0.27,10.43,0.32,4.03,0.42,202.89,26.03,1020.87,110.51
120,WWWWY,0,init,0,-10.92,0.75,-6.32,0.18,10.92,0.75,...,-6.61,0.16,10.30,0.46,3.69,0.49,213.74,39.42,938.44,60.20


In [28]:
data.to_csv("./FEN_HTVS_dataset.csv")